In [1]:
import pandas as pd
import numpy as np

In [ ]:
col = pd.read_csv('/Users/joamacha/Library/CloudStorage/OneDrive-TexasTechUniversity/Personal/Projects/Code/GitHub/ColonialLegacy/COLDAT_colonies.csv')
efw = pd.read_excel('/Users/joamacha/Library/CloudStorage/OneDrive-TexasTechUniversity/Personal/Projects/Code/GitHub/ColonialLegacy/efotw-2022-master-index-data-for-researchers-iso.xlsx')
vdem = pd.read_excel('/Users/joamacha/Library/CloudStorage/OneDrive-TexasTechUniversity/Personal/Projects/Code/GitHub/ColonialLegacy/VDem.xls')
vdem.rename(columns={'v2x_libdem':'libdem','v2xcl_prpty':'prop_rights','v2clstown':'stateown'}, inplace=True)

In [126]:
#Keep only colonies 
col = col.loc[col['time_total'] > 0]

In [128]:
# Dummy for approximations of State Ownership for colonies beggining 1789
for i in range(len(col)):
    if col.iloc[i,10:18].min() < 1789:
        col.iloc[i,37] = 1
    else:
        pass    
        


In [154]:
# Initialize columns with placeholder NaN
for colonizer in colonizers: 
    col[f'efw_{colonizer}'] = np.nan

In [218]:
#Names in V-Dem Data
cold = {'belgium':'Belgium', 'britain':'United Kingdom', 'france':'France','germany':'Germany','italy':'Italy','netherlands':'Netherlands','portugal':'Portugal','spain':'Spain'}

# Initialize column
col['avg_efw'] = np.nan

for country in col.loc[col['independence'] >= 1789, 'country_code']:
    for colonizer in colonizers: 
        end_year = col.loc[col['country_code']==country, f'colend_{colonizer}'].values[0]
        if end_year < 1789:
            avg_efw = vdem.loc[(vdem['country_name'] == cold[colonizer]) & (vdem['year'] == 1789), 'stateown'].values[0]
            timec = col.loc[col['country_code']==country, f'time_{colonizer}'].values[0]
            col.loc[col['country_code']==country, f'efw_{colonizer}'] = avg_efw * timec
        else: 
            begin_year = col.loc[col['country_code']==country, f'colstart_{colonizer}'].values[0]
            avg_efw = vdem.loc[(vdem['country_name'] == cold[colonizer]) & (vdem['year'] >= begin_year) & (vdem['year'] <= end_year), 'stateown'].mean() 
            timec = col.loc[col['country_code']==country, f'time_{colonizer}'].values[0]
            col.loc[col['country_code']==country, f'efw_{colonizer}'] = avg_efw * timec

col['avg_efw'] = col[col.columns[38:46]].sum(axis=1) / col['time_total']  

In [220]:
#Exclude 2020 because of the pandemic
efw = efw.loc[(efw['Year']<2020) & (efw['Year'] >1999)].rename(columns={'World Bank Region': "wb_region"})
efw.sort_values(by='Countries').head()

,Year,ISO_Code_2,ISO_Code_3,wb_region,"World Bank Current Income Classification, 1990-present (L=Low income, LM=Lower middle income, UM=Upper middle income, H=High income)",Countries,efw,Area 1,Area 2,Area 3,Area 4,Area 5,std
165,2019,AL,ALB,Europe & Central Asia,UM,Albania,7.789804,7.914268,5.258761,9.826940,8.230911,7.700060,1.641201
2970,2002,AL,ALB,Europe & Central Asia,LM,Albania,6.742276,8.223671,4.845155,7.101343,6.097055,7.291095,1.287959
1485,2011,AL,ALB,Europe & Central Asia,LM,Albania,7.520319,8.071567,5.132500,9.641108,6.996151,7.729106,1.644798
1980,2008,AL,ALB,Europe & Central Asia,LM,Albania,7.368699,8.225602,5.479977,9.433083,6.533912,7.121853,1.526479
330,2018,AL,ALB,Europe & Central Asia,UM,Albania,7.759843,7.811582,5.221942,9.760704,8.221900,7.767841,1.632527


In [221]:
means = efw.groupby('ISO_Code_3')['efw','Area 1','Area 2','Area 3','Area 4','Area 5','std'].mean()

<ipython-input-221-80b283be3827>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  means = efw.groupby('ISO_Code_3')['efw','Area 1','Area 2','Area 3','Area 4','Area 5','std'].mean()


In [222]:
df = pd.merge(col, means, left_on='country_code', right_on='ISO_Code_3', how='left')

In [ ]:
df = pd.read_excel()

In [223]:
df.to_excel('colonies.xlsx')

# WORLD VALUE SURVEY

In [8]:
wvs = pd.read_excel('/Users/joamacha/Library/CloudStorage/OneDrive-TexasTechUniversity/Personal/Projects/Code/GitHub/ColonialLegacy/WVS.xls')
df = pd.read_excel('/Users/joamacha/Library/CloudStorage/OneDrive-TexasTechUniversity/Personal/Projects/Code/GitHub/ColonialLegacy/colonies.xlsx')

In [9]:
df = pd.merge(df, wvs, left_on='country_code', right_on='COUNTRY_ALPHA', how='left')
df.to_excel('colonies_wvs.xlsx')